In [9]:
!nvidia-smi

Thu Apr 24 14:40:09 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100-SXM2-32GB           On  |   00000000:3B:00.0 Off |                    0 |
| N/A   32C    P0             41W /  300W |       1MiB /  32768MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

#### Clone Github Repository

In [ ]:
# !git clone https://github.com/sijiasiga/CLIP_video.git

In [11]:
# From CLIP
# !conda install --yes -c pytorch pytorch=1.7.1 torchvision cudatoolkit=11.0
# !pip install ftfy regex tqdm
# !pip install opencv-python boto3 requests pandas

#### Download Dataset

Download MSRVTT training data set

In [12]:
# # Comment if the file exits
# !wget -O /ocean/projects/cis250079p/shared/data/msrvtt_data.zip https://github.com/ArrowLuo/CLIP4Clip/releases/download/v0.0/msrvtt_data.zip
# !unzip /ocean/projects/cis250079p/shared/data/msrvtt_data.zip -d /ocean/projects/cis250079p/shared/data

Download MSRVTT raw video dataset

In [13]:
# !wget -O /ocean/projects/cis250079p/shared/data/msrvtt_raw_video_data.zip https://www.robots.ox.ac.uk/~maxbain/frozen-in-time/data/msrvtt_raw_video_data.zip
# !unzip /ocean/projects/cis250079p/shared/data/msrvtt_raw_video_data.zip -d /ocean/projects/cis250079p/shared/data

#### MSRVTT Data Path

In [14]:
DATA_PATH = "/ocean/projects/cis250079p/shared/data/msrvtt_data"  # Directory of MSRVTT data
VIDEO_PATH = "/ocean/projects/cis250079p/shared/data/msrvtt_raw_video_data/videos/all/"    # Directory of MSRVTT raw video data


#### Split the subset

In [15]:
import pandas as pd

# Load training dataset
train_path = f"{DATA_PATH}/MSRVTT_train.9k.csv"
df = pd.read_csv(train_path)

print("Size: ", len(df))
print("Column names", df.columns.tolist())
display(df.head())

# Choose a subset
def sample_subset(df, frac=1, random_state=42):
    return df.sample(frac=frac, random_state=random_state).reset_index(drop=True)

# Set fraction to 0.1
subset_df = sample_subset(df, frac=0.1)
print("Size of subset：", len(subset_df))
display(subset_df.head())


Size:  9000
Column names ['video_id']


,video_id
0,video0
1,video1
2,video2
3,video3
4,video4


Size of subset： 900


,video_id
0,video8405
1,video1162
2,video582
3,video4081
4,video9139


In [16]:
# # Save the subset
# subset_path = f"{DATA_PATH}/MSRVTT_train.subset.csv"
# subset_df.to_csv(subset_path, index=False)

# print("Subset CSV saved to:", subset_path)

In [17]:
import torch
print("Current device:", torch.cuda.get_device_name() if torch.cuda.is_available() else "CPU only")

Current device: Tesla V100-SXM2-32GB


#### Download CLIP-32

In [18]:
# # Download CLIP (ViT-B/32) weight
# !wget -P ./modules https://openaipublic.azureedge.net/clip/models/cc12fdd5df8b6c2c9118e754d5f1ec50b6706b3cd1a1e0dabfdfd5fcd8c38e38/ViT-B-32.pt

# # Download CLIP (ViT-B/32) weight
# !wget -P /ocean/projects/cis250079p/sma6/CLIP_video/modules https://openaipublic.azureedge.net/clip/models/cc12fdd5df8b6c2c9118e754d5f1ec50b6706b3cd1a1e0dabfdfd5fcd8c38e38/ViT-B-32.pt

In [19]:
torch.cuda.empty_cache()

In [20]:
# # Updata changes to files. Comment if the files are not changed
# !cd CLIP_video && git pull

In [21]:
# import os
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# !python /ocean/projects/cis250079p/sma6/CLIP_video/main_task_retrieval.py \
# --do_train \
# --num_thread_reader=0 \
# --epochs=5 \
# --batch_size=32 \
# --train_csv {DATA_PATH}/MSRVTT_train.subset.csv \
# --val_csv {DATA_PATH}/MSRVTT_JSFUSION_test.csv \
# --data_path {DATA_PATH}/MSRVTT_data.json \
# --features_path {VIDEO_PATH} \
# --output_dir /ocean/projects/cis250079p/sma6/CLIP_video/ckpts/ckpt_msrvtt_retrieval_looseType \
# --lr 1e-4 \
# --max_words 32 \
# --max_frames 8 \
# --batch_size_val 8 \
# --datatype msrvtt \
# --expand_msrvtt_sentences \
# --feature_framerate 1 \
# --coef_lr 1e-3 \
# --freeze_layer_num 0 \
# --slice_framepos 2 \
# --loose_type \
# --linear_patch 2d \
# --sim_header meanP \
# --pretrained_clip_name ViT-B/32 \
# --fp16 --fp16_opt_level O1


In [23]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

!python /ocean/projects/cis250079p/sma6/CLIP_video/main_task_retrieval.py \
--do_train \
--num_thread_reader=8 \
--epochs=5 \
--batch_size=64 \
--train_csv {DATA_PATH}/MSRVTT_train.subset.csv \
--val_csv {DATA_PATH}/MSRVTT_JSFUSION_test.csv \
--data_path {DATA_PATH}/MSRVTT_data.json \
--features_path {VIDEO_PATH} \
--output_dir /ocean/projects/cis250079p/sma6/CLIP_video/ckpts/ckpt_msrvtt_retrieval_looseType \
--lr 1e-4 \
--max_words 32 \
--max_frames 8 \
--batch_size_val 8 \
--datatype msrvtt \
--expand_msrvtt_sentences \
--feature_framerate 1 \
--coef_lr 1e-3 \
--freeze_layer_num 6 \
--slice_framepos 2 \
--loose_type \
--linear_patch 2d \
--sim_header tightTransf \
--pretrained_clip_name ViT-B/32 \
--fp16 \
--fp16_opt_level O1

04/24/2025 14:42:38 - INFO -   Effective parameters:
04/24/2025 14:42:38 - INFO -     <<< batch_size: 64
04/24/2025 14:42:38 - INFO -     <<< batch_size_val: 8
04/24/2025 14:42:38 - INFO -     <<< cache_dir: 
04/24/2025 14:42:38 - INFO -     <<< coef_lr: 0.001
04/24/2025 14:42:38 - INFO -     <<< cross_model: cross-base
04/24/2025 14:42:38 - INFO -     <<< cross_num_hidden_layers: 4
04/24/2025 14:42:38 - INFO -     <<< data_path: /ocean/projects/cis250079p/shared/data/msrvtt_data/MSRVTT_data.json
04/24/2025 14:42:38 - INFO -     <<< datatype: msrvtt
04/24/2025 14:42:38 - INFO -     <<< do_eval: False
04/24/2025 14:42:38 - INFO -     <<< do_lower_case: False
04/24/2025 14:42:38 - INFO -     <<< do_pretrain: False
04/24/2025 14:42:38 - INFO -     <<< do_train: True
04/24/2025 14:42:38 - INFO -     <<< epochs: 5
04/24/2025 14:42:38 - INFO -     <<< eval_frame_order: 0
04/24/2025 14:42:38 - INFO -     <<< expand_msrvtt_sentences: True
04/24/2025 14:42:38 - INFO -     <<< feature_framerate: